# KoSAIM 2023 Summer School - Transformers

## Topic: Diagnosis Code prediction from Discharge Summary

### Presentor: Junu Kim
- Contact: kjune0322@kaist.ac.kr

# Section 1. Setup

In [ ]:
# Download Data
# wget ~~

# Install Requirements

# !pip install transformers
# !pip install accelerate
# Import Libraries

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import pandas as pd
import torch
import numpy as np
import transformers
import matplotlib.pyplot as plt

# Section 2. Data Preprocessing

In [ ]:
import os
os.chdir('/home/data_storage/mimiciii-1.4/')

In [ ]:
# Make Labels

diagnoses = pd.read_csv('DIAGNOSES_ICD.csv')
diagnoses

In [ ]:
# Filter Columns
diagnoses = diagnoses[['HADM_ID', 'ICD9_CODE']]
diagnoses

In [ ]:
# Show Histogram of ICD9_CODE Frequency
diagnoses['ICD9_CODE'].value_counts().hist(
    bins=list(range(0, 1000, 10)),
    )
plt.title('Histogram of ICD9_CODE Frequency')
plt.xlabel('Frequency')
plt.ylabel('ICD9_CODE')

In [ ]:
# Filter ICD9 Codes Appearing More than 50 Times
counts = diagnoses['ICD9_CODE'].value_counts()
counts = counts[counts > 100]
print("Number of ICD9 Codes Appearing More than 100 Times: ", len(counts))
diagnoses = diagnoses[diagnoses['ICD9_CODE'].isin(counts.index)]
diagnoses

In [ ]:
# Encode Labels
label = pd.get_dummies(diagnoses['ICD9_CODE'])
label['HADM_ID'] = diagnoses['HADM_ID']
label = label.groupby('HADM_ID').sum().astype(bool).astype(int).reset_index()
label

In [ ]:
notes = pd.read_csv('NOTEEVENTS.csv')

# NOTE! This line is skipped
notes = notes[notes['CATEGORY'] == 'Discharge summary']
notes

In [ ]:
# Filter Columns
notes = notes[['HADM_ID', 'TEXT']]

# Join with Labels
data = notes.merge(label, on='HADM_ID', how='inner')
data

In [ ]:
# Shuffle and  Train, Validation, Test Split
from sklearn.model_selection import train_test_split

data = data.sample(frac=1, random_state=42)
train, valid = train_test_split(data, test_size=0.2, random_state=42)
valid, test = train_test_split(valid, test_size=0.5, random_state=42)

print("Length of Train: ", len(train))
print("Length of Valid: ", len(valid))
print("Length of Test: ", len(test))
train

# Section 3. Build Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=874, problem_type="multi_label_classification")
model

In [ ]:
# Train only CLS Header
model.requires_grad_(False)
model.classifier.requires_grad_(True)

In [ ]:
# Make Dataset and Dataloader

from torch.utils.data import Dataset, DataLoader

class NoteData(Dataset):
    def __init__(self, data, tokenizer):
        super().__init__()
        self.data = data
        self.tokenizer = tokenizer
    
    def __len__(self):
        # return 100
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['TEXT']
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=True,
            truncation=True,
            return_tensors='pt',
            )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'labels': torch.tensor(np.asarray(row[2:].values, dtype=np.int32), dtype=torch.float),
            }


In [ ]:
train_dataset = NoteData(train, tokenizer)
valid_dataset = NoteData(valid, tokenizer)
test_dataset = NoteData(test, tokenizer)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = torch.sigmoid(torch.tensor(logits)).detach().numpy()
    return {
        "auroc": roc_auc_score(labels, preds, average='macro', multi_class='ovr'),
        "auprc": average_precision_score(labels, preds, average='macro'),
    }

In [ ]:
from transformers import Trainer, TrainingArguments

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,

)

In [ ]:
# Train Model
trainer.train()

In [ ]:
# Test Model
pred = trainer.predict(test_dataset)